In [448]:

import pandas as pd
import pyodbc
import numpy as np

from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer
from feature_engine.encoding import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, precision_score, accuracy_score
from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer
from feature_engine.encoding import OneHotEncoder


import warnings



warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=Warning, module='feature_engine')
import logging

# Defina o nível de log para suprimir mensagens informativas
logging.getLogger('lightgbm').setLevel(logging.WARNING)



In [449]:
# Configurações de conexão com o servidor SQL
server = 'xxxx'  # Seu servidor SQL
database = 'xxxx'  # Seu banco de dados
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
# Conectando ao servidor SQL
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [450]:
sql_query = '''
SELECT chain_id, dex_id, pair_address, token_address, token_name, token_symbol, token_m5_buys, token_m5_sells, token_h1_buys, token_h1_sells, token_h1_to_m5_buys, token_liquidity, token_market_cap, token_created_since_horas, token_created_since_min, token_eti, token_header, token_website, token_twitter, token_links, token_img_key, token_price_usd,  token_price_change_h24, token_price_change_h6, token_price_change_h1, token_price_change_m5, token_created_dia, token_created_data, token_created_hora

FROM DEX;
'''

In [451]:
df = pd.read_sql(sql_query, conn)

In [570]:
# Substituir valores diferentes de '0' por '1'
df['token_img_key'] = df['token_img_key'].apply(lambda x: '1' if x != '0' else x)
print(df['token_img_key'].value_counts())

token_img_key
0    2175961
1    1517613
Name: count, dtype: int64


In [571]:
# Primeira alteração: substituir valores na coluna 'dex_id' com mais de 20 caracteres por "outros"
df.loc[df['dex_id'].str.len() > 20, 'dex_id'] = 'outros'


# Terceira alteração: substituir valores NULL na coluna 'token_h1_to_m5_buys' por 0
df['token_h1_to_m5_buys'].fillna(0, inplace=True)



data ot


In [572]:
data_limite = pd.to_datetime('2024-04-09')

In [573]:
df['token_created_data'] = pd.to_datetime(df['token_created_data'], format='%Y%m%d')
# Defina a data limite para a segmentação




horaaa

In [658]:
hora_seg1 = 15
hora_seg2 = hora_seg1 + 15
hora_seg3 = hora_seg1 + 60
hora_seg4 = hora_seg2  + 60
hora_limite = hora_seg4 +  10

Segmentando a base

In [659]:
df_1hora = df[(df['token_created_since_min'] >= hora_seg1) & (df['token_created_since_min'] <= hora_seg2)]

# Agrupar os dados pela coluna 'token_address' e manter apenas a linha com o maior valor de 'token_created_since_horas' em cada grupo
df_1hora = df_1hora.loc[df_1hora.groupby('token_address')['token_created_since_min'].idxmax()]



In [660]:
df_limite = df[(df['token_created_since_min'] >= hora_limite) & (df['token_created_since_min'] <= (hora_limite + 90))]

# Agrupar os dados pela coluna 'token_address' e manter apenas a linha com o maior valor de 'token_created_since_horas' em cada grupo
df_limite = df_limite.loc[df_limite.groupby('token_address')['token_created_since_min'].idxmax()]

In [661]:
print(df_1hora['token_address'].nunique())

41846


segmentando nova hora

In [662]:
df_2hora = df[(df['token_created_since_min'] >= hora_seg3) & (df['token_created_since_min'] <=  hora_seg4)]

# Agrupar os dados pela coluna 'token_address' e manter apenas a linha com o maior valor de 'token_created_since_horas' em cada grupo
df_2hora = df_2hora.loc[df_2hora.groupby('token_address')['token_created_since_min'].idxmax()]



In [663]:
print(df_2hora['pair_address'].nunique())

25572


merged

In [664]:
df_novomerged = pd.merge(df_1hora, df_limite[['token_address', 'token_price_usd', 'token_liquidity', 'token_created_since_min']], on='token_address', how='left')

# Preencha os valores faltantes com 0
df_novomerged['token_price_usd_y'].fillna(0, inplace=True)
df_novomerged['token_liquidity_y'].fillna(0, inplace=True)
df_novomerged['token_created_since_min_y'].fillna(0, inplace=True)
df_novomerged.rename(columns={'token_price_usd_y': 'token_price_usd_fut', 'token_liquidity_y': 'token_liquidity_fut','token_created_since_min_y':'token_created_since_min_fut', 
 'token_price_usd_x': 'token_price_usd', 'token_liquidity_x': 'token_liquidity','token_created_since_min_x':'token_created_since_min'                            }, inplace=True)



In [665]:
df_merged = pd.merge(df_novomerged, df_2hora[['token_address', 'token_price_usd', 'token_liquidity', 'token_created_since_min' ]], on='token_address', how='left')
# Preencha os valores faltantes com 0
df_merged['token_price_usd_y'].fillna(0, inplace=True)
df_merged['token_liquidity_y'].fillna(0, inplace=True)
df_merged['token_created_since_min_y'].fillna(0, inplace=True)

In [666]:
print(df_merged.columns.tolist())

['chain_id', 'dex_id', 'pair_address', 'token_address', 'token_name', 'token_symbol', 'token_m5_buys', 'token_m5_sells', 'token_h1_buys', 'token_h1_sells', 'token_h1_to_m5_buys', 'token_liquidity_x', 'token_market_cap', 'token_created_since_horas', 'token_created_since_min_x', 'token_eti', 'token_header', 'token_website', 'token_twitter', 'token_links', 'token_img_key', 'token_price_usd_x', 'token_price_change_h24', 'token_price_change_h6', 'token_price_change_h1', 'token_price_change_m5', 'token_created_dia', 'token_created_data', 'token_created_hora', 'token_price_usd_fut', 'token_liquidity_fut', 'token_created_since_min_fut', 'token_price_usd_y', 'token_liquidity_y', 'token_created_since_min_y']


In [667]:
print(df_merged['pair_address'].nunique())

41846


In [668]:
df_merged['crescimento_price'] = (df_merged['token_price_usd_y'] / df_merged['token_price_usd_x']) - 1
df_merged['crescimento_liquidity'] = (df_merged['token_liquidity_y'] / df_merged['token_liquidity_x']) - 1


In [669]:
condicao = (df_merged['crescimento_price'] > 0.046) & (df_merged['crescimento_liquidity'] > 0) & (df_merged['token_price_usd_fut'] > df_merged['token_price_usd_x']) & (df_merged['token_liquidity_fut'] > 100)
df_merged['crescimento_flag'] = np.where(condicao, 1, 0)


In [670]:
df_diferenca = df_merged[df_merged['crescimento_flag'] == 1]
summary = df_diferenca['crescimento_price'].describe()
print(summary)

count    5.830000e+03
mean     1.232600e+25
std      9.411445e+26
min      4.613413e-02
25%      2.215275e-01
50%      6.266547e-01
75%      1.684822e+00
max      7.186056e+28
Name: crescimento_price, dtype: float64


In [671]:
removerbase = (df_merged['crescimento_price'] == -1) & (df_merged['token_liquidity_fut'] > 100)

# Removendo linhas que satisfazem a condição
df_merged = df_merged.drop(df_merged[removerbase].index)

In [672]:
print(df_merged['pair_address'].nunique())

38954


In [673]:
# Segmentar o DataFrame com base na data limite
df_ot = df_merged[df_merged['token_created_data'] >= data_limite]
df_otnew = df_merged[(df_merged['token_created_data'] >= data_limite) & (df_merged['token_h1_buys'] >20)]
df_merged = df_merged[df_merged['token_created_data'] < data_limite]


In [674]:
print(df_merged['pair_address'].nunique())

33771


categorizando variaveis

In [675]:
cat_vars =['chain_id', 'dex_id','token_created_dia', 'token_website', 'token_twitter', 'token_eti', 'token_header', 'token_img_key', 'token_created_hora' ]
target = ['crescimento_flag']
num_vars = ['token_m5_buys','token_m5_sells', 'token_h1_buys', 'token_h1_sells', 'token_liquidity_x', 'token_market_cap', 'token_links', 'token_price_change_h1', 'token_price_change_m5', 'token_h1_to_m5_buys']

features = num_vars+ cat_vars
df[cat_vars] = df[cat_vars].astype('category')





X = df_merged[cat_vars + num_vars]
y= df_merged[target]
X_ot = df_ot[cat_vars + num_vars]
y_ot= df_ot[target]
X_otnew = df_otnew[cat_vars + num_vars]
y_otnew= df_otnew[target]

# Converta as colunas categóricas para o tipo 'category'
X[cat_vars] = X[cat_vars].astype('category')
X[cat_vars] = X[cat_vars].astype('category')



C:\Users\xavie\AppData\Local\Temp\ipykernel_18968\762259577.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_vars] = X[cat_vars].astype('category')
C:\Users\xavie\AppData\Local\Temp\ipykernel_18968\762259577.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_vars] = X[cat_vars].astype('category')


In [676]:
num_linhas = len(df_ot)
print("Número de linhas:", num_linhas)


Número de linhas: 5183


treino e teste

RF

In [677]:
rf = Pipeline(steps=[
    ("numeric_imputer", ArbitraryNumberImputer(variables=num_vars, arbitrary_number=-999)),
    ("categoric_imputer", CategoricalImputer(variables=cat_vars, fill_value="missing")),
    ("one", OneHotEncoder(variables=cat_vars)),
    ("rf", RandomForestClassifier(random_state=42))
])

# Ajuste o pipeline aos seus dados de treinamento (X_train são suas features e y_train são seus rótulos)
rf.fit(X, y)

Pipeline(steps=[('numeric_imputer',
                 ArbitraryNumberImputer(arbitrary_number=-999,
                                        variables=['token_m5_buys',
                                                   'token_m5_sells',
                                                   'token_h1_buys',
                                                   'token_h1_sells',
                                                   'token_liquidity_x',
                                                   'token_market_cap',
                                                   'token_links',
                                                   'token_price_change_h1',
                                                   'token_price_change_m5',
                                                   'token_h1_to_m5_buys'])),
                ('categoric_imputer',
                 CategoricalImputer(fill_value='missing',
                                    variab...d', 'dex_id',
                                               'token_created_dia',
                                               'token_website', 'token_twitter',
                                               'token_eti', 'token_header',
                                               'token_img_key',
                                               'token_created_hora'])),
                ('one',
                 OneHotEncoder(variables=['chain_id', 'dex_id',
                                          'token_created_dia', 'token_website',
                                          'token_twitter', 'token_eti',
                                          'token_header', 'token_img_key',
                                          'token_created_hora'])),
                ('rf', RandomForestClassifier(random_state=42))])

In [678]:
scores = cross_val_score(rf, X, y, cv=5)
precision = precision_score(y, rf.predict(X))
accuracy = accuracy_score(y, rf.predict(X))
print("Precisão:", precision)
print("Acurácia:", accuracy)
print(scores)

Precisão: 1.0
Acurácia: 0.9999407775902401
[0.88364175 0.88081137 0.88688185 0.85075511 0.84690554]


In [679]:
from sklearn.metrics import roc_auc_score
probas_pred = rf.predict_proba(X_ot)[:, 1]

# Ajuste do limiar de decisão (por exemplo, 0.5)
limiar = 0.55

# Classificação com base no limiar
previsoes_rf = (probas_pred >= limiar).astype(int)

# Calcular a precisão com o novo limiar
precision = precision_score(y_ot, previsoes_rf)
print("precision",precision)

accuracy = accuracy_score(y_ot, previsoes_rf)
print("Acurácia:", accuracy)

roc_auc_teste = roc_auc_score(y_ot, probas_pred)
print("ROC AUC:", roc_auc_teste)

quantidade_rf2 = np.sum(previsoes_rf == 1)
print("Quantidade classificada como 1:", quantidade_rf2)

precision 0.7590361445783133
Acurácia: 0.8898321435462088
ROC AUC: 0.8105424703270333
Quantidade classificada como 1: 166


In [680]:
probas_pred = rf.predict_proba(X_otnew)[:, 1]

# Ajuste do limiar de decisão (por exemplo, 0.5)
limiar = 0.55

# Classificação com base no limiar
previsoes_rf = (probas_pred >= limiar).astype(int)

# Calcular a precisão com o novo limiar
precision = precision_score(y_otnew, previsoes_rf)
print("precision",precision)

accuracy = accuracy_score(y_otnew, previsoes_rf)
print("Acurácia:", accuracy)

roc_auc_teste = roc_auc_score(y_otnew, probas_pred)
print("ROC AUC:", roc_auc_teste)

quantidade_rf2 = np.sum(previsoes_rf == 1)
print("Quantidade classificada como 1:", quantidade_rf2)

precision 0.7835051546391752
Acurácia: 0.8835842820418656
ROC AUC: 0.7930776425497271
Quantidade classificada como 1: 97


ganhos


In [681]:
limiar = 0.55
X_otcompleto = X_otnew.copy()
colunas_token = ['token_name', 'token_address', 'pair_address', 'token_symbol', 'token_created_data', 'token_created_hora',  'crescimento_flag', 'crescimento_price', 'token_price_usd_x', 'token_price_usd_y', 'token_price_usd_fut', 'token_created_since_min_x', 'token_created_since_min_y', 'token_liquidity_y', 'token_liquidity_fut']
X_otcompleto[colunas_token] = df_ot[colunas_token]
X_otcompleto['previsoes'] = previsoes_rf  # Corrigir o nome da variável
X_otcompleto['proba'] = probas_pred
filtro = probas_pred > limiar
X_otcompleto= X_otcompleto[filtro]


# Calcular a soma da coluna 'crescimento' para cada categoria 'dex_id'
contagem_e_soma_por_dex_idnew = X_otcompleto.groupby('chain_id').agg({'crescimento_price': 'sum', 'chain_id': 'size'})

# Renomear a coluna 'dex_id' para 'contagem'
contagem_e_soma_por_dex_idnew.rename(columns={'chain_id': 'contagem'}, inplace=True)

# Exibir a contagem e a soma por categoria

print(contagem_e_soma_por_dex_idnew)


                 crescimento_price  contagem
chain_id                                    
aptos                     0.000000         0
arbitrum                 10.516744        10
astar                     0.000000         0
avalanche                 4.973757         8
base                     53.496471        22
beam                      0.000000         0
blast                     2.926658         3
bsc                      96.818832        14
canto                     0.000000         0
celo                      0.000000         0
conflux                   0.000000         0
core                      0.000000         0
cronos                    0.000000         0
degenchain               10.479304         8
dogechain                 0.000000         0
elastos                   0.000000         0
ethereum                 78.286267        24
ethereumclassic           0.000000         0
ethereumpow               0.000000         0
fantom                    0.544464         1
filecoin  

In [682]:
df_filtrado = X_otcompleto[X_otcompleto['chain_id'] == 'bsc']
df_filtrado = df_filtrado[['token_name', 'pair_address' ,'proba']]

print(df_filtrado)

         token_name                                pair_address  proba
9240     BlockGames  0xeF2440df47521647FC997c3c99a04C1297707053   0.59
11178   Piggy Bonki  0x94A6CD8567d547079Baba34Ab63ad8B013180791   0.63
13336          Saga  0x191110a96429cA2Ec9e796D7e34ca86a730D1abd   0.63
13714          FOXY  0xCB9481B0B4a4f3aBacafCBE77AA0d6Eeb400BA72   0.65
17234          Foxy  0x913EC7732271bc98aE4bF4070B641cad07eD6f51   0.65
20254     Shiva Inu  0xcbAfceCc2b83513F0F276c43CA049D9D482817b5   0.68
21296        Ordify  0x5a7eFA776CaC8E6d836b0D3aa0d1C4Aba6b01e04   0.56
21608          Fish  0xBa3623b05b20fc04d30731dF0475F980E20B4924   0.88
22018    World Coin  0x2a2214A59Fc9227d9c9852c04c9B4dCE5659EeE3   0.86
22303      Football  0x85b03A9a84BA6614Ff53a9E7ee1C8BFa2bFB6C65   0.80
23000    Gelly Dons  0x41ac3449263d84e17c5cc85e99E6Ba5f68d30427   0.56
25063       Context  0xCAf7307a587A135343d7AF1ebA86b58E9A3F9448   0.71
26606  Book Of Bull  0x3dd025B315354F742f2F126f9afe0B080b8A9BD9   0.63
27030 

In [683]:
print(X_otcompleto)

         chain_id     dex_id token_created_dia token_website token_twitter  \
598    degenchain  degenswap      quinta-feira         False         False   
843      arbitrum  sushiswap      quinta-feira         False         False   
903    degenchain  degenswap       terça-feira         False         False   
1211     arbitrum  sushiswap      quarta-feira         False         False   
1936     arbitrum    camelot      quinta-feira         False         False   
...           ...        ...               ...           ...           ...   
27082    ethereum    uniswap      quarta-feira         False         False   
27464        base    uniswap      quarta-feira         False          True   
27565        base    uniswap       terça-feira         False         False   
27630    ethereum    uniswap       terça-feira         False         False   
35964      solana    raydium      quinta-feira          True          True   

      token_eti token_header token_img_key token_created_hora  

x completo


df 10 min

In [684]:
sql_query10 = '''
SELECT chain_id, dex_id, pair_address, token_address, token_name, token_symbol, token_m5_buys, token_m5_sells, token_h1_buys, token_h1_sells, token_h1_to_m5_buys, token_liquidity, token_market_cap, token_created_since_horas, token_created_since_min, token_eti, token_header, token_website, token_twitter, token_links, token_img_key, token_price_usd,  token_price_change_h24, token_price_change_h6, token_price_change_h1, token_price_change_m5, token_created_dia, token_created_data, token_created_hora

FROM dex10;
'''

In [ ]:
df10 = pd.read_sql(sql_query, conn)

In [ ]:
print(df10.columns.tolist())

['chain_id', 'dex_id', 'pair_address', 'token_address', 'token_name', 'token_symbol', 'token_m5_buys', 'token_m5_sells', 'token_h1_buys', 'token_h1_sells', 'token_h1_to_m5_buys', 'token_liquidity', 'token_market_cap', 'token_created_since_horas', 'token_created_since_min', 'token_eti', 'token_header', 'token_website', 'token_twitter', 'token_links', 'token_img_key', 'token_price_usd', 'token_price_change_h24', 'token_price_change_h6', 'token_price_change_h1', 'token_price_change_m5', 'token_created_dia', 'token_created_data', 'token_created_hora']


In [ ]:
# Substituir valores diferentes de '0' por '1'
df10['token_img_key'] = df10['token_img_key'].apply(lambda x: '1' if x != '0' else x)
print(df10['token_img_key'].value_counts())

token_img_key
0    2131448
1    1487389
Name: count, dtype: int64


In [ ]:
# Primeira alteração: substituir valores na coluna 'dex_id' com mais de 20 caracteres por "outros"
df10.loc[df10['dex_id'].str.len() > 20, 'dex_id'] = 'outros'


# Terceira alteração: substituir valores NULL na coluna 'token_h1_to_m5_buys' por 0
df10['token_h1_to_m5_buys'].fillna(0, inplace=True)

In [ ]:
df10['token_created_data'] = pd.to_datetime(df10['token_created_data'], format='%Y%m%d')

In [ ]:
print(df10)

        chain_id       dex_id  \
0           base      uniswap   
1         zksync  pancakeswap   
2           base      uniswap   
3         fantom   spookyswap   
4         solana      raydium   
...          ...          ...   
3618832      ton       dedust   
3618833   solana      raydium   
3618834   solana      raydium   
3618835     base      uniswap   
3618836     base    sushiswap   

                                             pair_address  \
0              0x5Ab85Ec3D5241452900212aEc7fc46254F8B4b3D   
1              0x97FE49f62ecAE19F6635e9Bd6F64A47bed133324   
2              0xa9Eb27856ab7f2527F6e650f117D472b20D653c2   
3              0xEdf34DAAF562aa39b7424A3cb97D4193c21C9456   
4            GFKBifF8zbUf52ZqARZtsfuTwX8fWsujtQqTuWW3Ecmy   
...                                                   ...   
3618832  EQDGdzOZIReQt_nU0FYbZo-5pB50c42KgJPicpdIiCSzo3Pw   
3618833      AGtkTfmACubX7yXTSv3G8eGvCxZiSq2Ck6ot3sWveM5Y   
3618834      3APG5BTBmQsNCuJqK2rsVYSmZsAoNX7Zaa7fxbTBF

In [ ]:
df10n = df10.copy()

In [ ]:
df10n['interval'] = pd.cut(df['token_created_since_min'], bins=range(0, 300, 20), right=False)

In [ ]:
df10n_filtered = df10n.loc[df10n.groupby(['pair_address', 'interval'], observed=True)['token_created_since_min'].idxmax()]



In [ ]:
print(df10n_filtered)

        chain_id     dex_id  \
1049422    venom  web3world   
1051618    venom  web3world   
1056267    venom  web3world   
1057674    venom  web3world   
1064629    venom  web3world   
...          ...        ...   
642910    solana    raydium   
644740    solana    raydium   
404919    solana    raydium   
368096    solana    raydium   
1910953   solana    raydium   

                                              pair_address  \
1049422  0:0121786b5319f6cc1b380e3d4f17e28273fc37fac500...   
1051618  0:0121786b5319f6cc1b380e3d4f17e28273fc37fac500...   
1056267  0:0121786b5319f6cc1b380e3d4f17e28273fc37fac500...   
1057674  0:0121786b5319f6cc1b380e3d4f17e28273fc37fac500...   
1064629  0:0121786b5319f6cc1b380e3d4f17e28273fc37fac500...   
...                                                    ...   
642910         zn1GSbRZbsL1bXRKhUmCc97TnXQoJfV5bEH1sD9SCsz   
644740         zn1GSbRZbsL1bXRKhUmCc97TnXQoJfV5bEH1sD9SCsz   
404919         zpy18s5sLo26hPDmzJRmw7P2RZ3whuG2ctAG9Ra5ddw   
368096 

In [ ]:
print(df10n_filtered.columns)

Index(['chain_id', 'dex_id', 'pair_address', 'token_address', 'token_name',
       'token_symbol', 'token_m5_buys', 'token_m5_sells', 'token_h1_buys',
       'token_h1_sells', 'token_h1_to_m5_buys', 'token_liquidity',
       'token_market_cap', 'token_created_since_horas',
       'token_created_since_min', 'token_eti', 'token_header', 'token_website',
       'token_twitter', 'token_links', 'token_img_key', 'token_price_usd',
       'token_price_change_h24', 'token_price_change_h6',
       'token_price_change_h1', 'token_price_change_m5', 'token_created_dia',
       'token_created_data', 'token_created_hora', 'interval'],
      dtype='object')


In [ ]:
df10n_filtered.columns

Index(['chain_id', 'dex_id', 'pair_address', 'token_address', 'token_name',
       'token_symbol', 'token_m5_buys', 'token_m5_sells', 'token_h1_buys',
       'token_h1_sells', 'token_h1_to_m5_buys', 'token_liquidity',
       'token_market_cap', 'token_created_since_horas',
       'token_created_since_min', 'token_eti', 'token_header', 'token_website',
       'token_twitter', 'token_links', 'token_img_key', 'token_price_usd',
       'token_price_change_h24', 'token_price_change_h6',
       'token_price_change_h1', 'token_price_change_m5', 'token_created_dia',
       'token_created_data', 'token_created_hora', 'interval'],
      dtype='object')

In [ ]:
pivot_table = df10n_filtered.pivot_table(index='pair_address', columns='interval', values=  'token_price_usd',  aggfunc='first')

In [ ]:
pivot_table.reset_index(inplace=True)

In [ ]:
pivot_table.columns = [f'{interval.left} a {interval.right} min' if isinstance(interval, pd.Interval) else interval for interval in pivot_table.columns]

In [ ]:
print(pivot_table)

                                            pair_address    0 a 20 min  \
0      0:0121786b5319f6cc1b380e3d4f17e28273fc37fac500...  3.136000e-03   
1      0:01269803fe53bd1fe40c02bc04a906c14c6efd38e6ee...  3.578000e-05   
2      0:01c00bdd6ed2ceb7b0adf8c8ea5a1857983bbbf0d4b1...           NaN   
3      0:03c2eca2cf727d710c2f7479ead0f66d10ae543306ef...           NaN   
4      0:06d2d46c135817293e85a7aa4fe2c458ec6fe47a6651...           NaN   
...                                                  ...           ...   
72569        ziav4qYc43wruyyp8mnLL4x2XkdWcNkV7cSGxqJ3XGk  3.631000e-03   
72570        zn1GSbRZbsL1bXRKhUmCc97TnXQoJfV5bEH1sD9SCsz  1.713000e-05   
72571        zpy18s5sLo26hPDmzJRmw7P2RZ3whuG2ctAG9Ra5ddw  1.111000e-08   
72572        zu6pfRUS57weJtJRKq4vogm3rZqdG1S7quqZfqiruZR  1.173000e-05   
72573        zvesNcpqGHSFHUEq3Jy9uNLx4HTH751euSDEm4MCCPD  5.253000e-04   

        20 a 40 min   40 a 60 min   60 a 80 min  80 a 100 min  100 a 120 min  \
0      3.049000e-03  3.049000e-

In [ ]:
pivot_table.columns

Index(['pair_address', '0 a 20 min', '20 a 40 min', '40 a 60 min',
       '60 a 80 min', '80 a 100 min', '100 a 120 min', '120 a 140 min',
       '140 a 160 min', '160 a 180 min', '180 a 200 min', '200 a 220 min',
       '220 a 240 min', '240 a 260 min', '260 a 280 min'],
      dtype='object')

In [ ]:
df_10min = pd.merge(X_otcompleto,pivot_table[pivot_table.columns], on='pair_address', how='left')

In [ ]:
print(df_10min)

     chain_id       dex_id token_created_dia token_website token_twitter  \
0    arbitrum    sushiswap      quarta-feira         False         False   
1        base      uniswap      quarta-feira         False          True   
2    ethereum      uniswap      quarta-feira         False         False   
3       blast     thruster      quarta-feira         False         False   
4   avalanche    traderjoe       terça-feira         False         False   
5        base      uniswap       terça-feira         False          True   
6        base      uniswap      quarta-feira         False          True   
7        base      uniswap       terça-feira         False          True   
8         bsc  pancakeswap      quarta-feira         False         False   
9    arbitrum      camelot      quarta-feira         False         False   
10        bsc  pancakeswap       terça-feira         False         False   
11  avalanche    traderjoe       terça-feira         False         False   
12  avalanch

In [ ]:
#usando API para verificar possível malicia no token
from goplus.token import Token

data = Token(access_token=None).token_security(
    chain_id="8453", addresses=["0xAD5451aA2692513e066f42bE8a469D40c8E578FA"]
)
print(data)